In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
## 우선 MovieLens data에 관해 모델 test
## CML은 implict한 data에 관한 식
## rate 점수가 4이상인 것에 관해서 진행(as positive) + 10 rating 이상인 것에 관해
## train/validation/test -> 60, 20 20%로 나누기

# Data 전처리(movielens)

In [9]:
%cd /content/drive/Othercomputers/notebook/2023-summer-internship/week3/

/content/drive/Othercomputers/notebook/2023-summer-internship/week3


In [10]:
pwd

'/content/drive/Othercomputers/notebook/2023-summer-internship/week3'

In [12]:
import pandas as pd
import sklearn
import numpy as np
import torch
import matplotlib.pyplot as plt

In [22]:
df = pd.read_csv('../data/ml-100k/u.data', sep='\t', header = None)
df.columns = ["user_id", "item_id", "rating", "timestamp"]
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [23]:
df.shape

(100000, 4)

In [24]:
## df[rating >= 4] = 1, df[rating < 4] = 0으로 변경
# Change the rating values of 4 or higher to 1, o.w 0
df.loc[df['rating'] < 4, 'rating'] = 0
df.loc[df['rating'] >= 4, 'rating'] = 1
# Display the updated DataFrame
df.head()

,user_id,item_id,rating,timestamp
0,196,242,0,881250949
1,186,302,0,891717742
2,22,377,0,878887116
3,244,51,0,880606923
4,166,346,0,886397596


In [27]:
count_1 = (df['rating'] == 1).sum()
count_0 = (df['rating'] == 0).sum()
print(count_1, count_0)

55375 44625


In [33]:
df.groupby(["user_id"])[["rating"]].count()

,rating
user_id,
1,272
2,62
3,54
4,24
5,175
...,...
939,49
940,107
941,22


In [36]:
# Filter user_ids with more than 10 ratings with a value of 1
filtered_df = df.groupby('user_id').filter(lambda x: x['rating'].sum() >= 10)

# Display the filtered DataFrame
filtered_df.head()

,user_id,item_id,rating,timestamp
0,196,242,0,881250949
1,186,302,0,891717742
2,22,377,0,878887116
3,244,51,0,880606923
4,166,346,0,886397596


In [39]:
filtered_df.shape

(98856, 4)

# CML모델 구현